<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="#Prepare-the-dataset" data-toc-modified-id="Prepare-the-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare the dataset</a></span></li></ul></div>

# Load libraries

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Prepare the dataset

In [2]:
data_dir = "./data/"

In [3]:
audio_pairs_df = pd.read_csv(data_dir + "audio_sentence_pairs_test.csv")
audio_pairs_df

,student_I_id,article_I_id,sentence_I_id,student_II_id,article_II_id,sentence_II_id
0,17021306_PhamCongNam,9,7,17021306_PhamCongNam,6,20
1,17021281_NguyenTrongLam,2,10,17021281_NguyenTrongLam,7,18
2,17021298_NguyenDinhNhatMinh,5,6,17021298_NguyenDinhNhatMinh,2,43
3,17021332_LeMinhTam,3,2,17021298_NguyenDinhNhatMinh,3,7
4,17021311_CaoMinhNhat,5,21,17021311_CaoMinhNhat,0,9
...,...,...,...,...,...,...
9995,17021311_CaoMinhNhat,6,18,17021306_PhamCongNam,9,12
9996,17021311_CaoMinhNhat,0,9,18020017_TranManhHieu,1,6
9997,18020833_NguyenKimLong,2,37,18020833_NguyenKimLong,7,0
9998,17021332_LeMinhTam,5,1,17021311_CaoMinhNhat,6,0


In [4]:
import _pickle as cPickle
import preprocessing

MFCC_LEN_LIM = 700

waveform_dir = data_dir + "waveform/"

def create_mfcc_data(student_id, article_id, sentence_id):
    audio_sentence = np.load(waveform_dir + student_id + "/{:02d}/".format(article_id + 1) + str(sentence_id) + ".npy")
    return preprocessing.mfcc(audio_sentence)[:MFCC_LEN_LIM]

data_mfcc = {
    "sentence_I": [],
    "sentence_II": [],
    "same_speaker": []
}

audio_pairs = audio_pairs_df.to_numpy()
print('Progress: ', end='')
for i in range(len(audio_pairs)):
    row = audio_pairs[i]
    data_mfcc['sentence_I'].append(create_mfcc_data(row[0], row[1], row[2]))
    data_mfcc['sentence_II'].append(create_mfcc_data(row[3], row[4], row[5]))
    data_mfcc['same_speaker'].append(row[0] == row[3])
    if i * 100 // len(audio_pairs) > (i - 1) * 100 // len(audio_pairs):
        print('|', end='')
    
cPickle.dump(data_mfcc, open(data_dir + f"dataset/data_mfcc_10.pkl", 'wb'), protocol=2)

Progress: ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||